In [2]:
import os

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
import json

In [5]:
raw=os.getenv("API_KEYS")
raw

'GROQ_API_KEYy'

In [8]:
import json

s = '{"language": "Python", "version": 3.11}'

p = json.loads(s)

print(p)

{'language': 'Python', 'version': 3.11}


In [4]:
os.getcwd()

'd:\\ecomm-prod-assistant\\notebook'

In [11]:
vstore=None

In [12]:
if not vstore:

    print("ok")

ok
